In [ ]:
!pip install pandas hazm scikit-learn torch transformers numpy flask flask_ngrok pyngrok

In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

import pandas as pd
from hazm import Normalizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel, AutoModelForQuestionAnswering
import torch
import numpy as np
from google.colab import files

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)

In [ ]:
files.upload()

In [ ]:
df = pd.read_csv('concat.csv')

In [ ]:
normalizer = Normalizer()
df['combined_text'] = df['combined_text'].apply(normalizer.normalize)

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['combined_text'])

In [ ]:
model_name_qa = "SajjadAyoubi/xlm-roberta-large-fa-qa"
tokenizer_qa = AutoTokenizer.from_pretrained(model_name_qa)
model_qa = AutoModelForQuestionAnswering.from_pretrained(model_name_qa)

In [ ]:
model_name_sim = "HooshvareLab/bert-fa-base-uncased"
tokenizer_sim = AutoTokenizer.from_pretrained(model_name_sim)
model_sim = AutoModel.from_pretrained(model_name_sim)

In [ ]:
def find_most_similar_docs(question, keywords, top_n=5):
    question_with_keywords = f"{question} {' '.join(keywords)}"
    question_vec = vectorizer.transform([normalizer.normalize(question_with_keywords)])
    similarities = cosine_similarity(question_vec, X).flatten()
    top_docs_indices = similarities.argsort()[-top_n:][::-1]
    return df.iloc[top_docs_indices]


In [ ]:
def get_answer(question, context, max_length=512):
    inputs = tokenizer_qa.encode_plus(
        question,
        context,
        add_special_tokens=True,
        return_tensors="pt",
        max_length=max_length,
        truncation=True
    )
    outputs = model_qa(**inputs)
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1

    answer = tokenizer_qa.convert_tokens_to_string(tokenizer_qa.convert_ids_to_tokens(inputs.input_ids[0][answer_start:answer_end]))
    return answer.strip()

In [ ]:
def semantic_similarity(sentence1, sentence2):
    inputs = tokenizer_sim(sentence1, sentence2, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model_sim(**inputs)

    embeddings1 = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    embeddings2 = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

    similarity = np.dot(embeddings1, embeddings2) / (np.linalg.norm(embeddings1) * np.linalg.norm(embeddings2))
    return similarity

In [ ]:
@app.route('/get_best_answer', methods=['POST'])
def get_best_answer():
    data = request.get_json()
    question = data.get('question')
    keywords = data.get('keywords', [])
    print(keywords)

    similar_docs = find_most_similar_docs(question, keywords, top_n=5)
    print(similar_docs)
    filtered_docs_df = pd.DataFrame(similar_docs)

    if not filtered_docs_df.empty:
        docs_to_evaluate = filtered_docs_df
    else:
        docs_to_evaluate = similar_docs

    best_answer = ""
    best_score = 0.0
    min_score_threshold = 0.00005
    min_answer_length = 1

    for idx, row in docs_to_evaluate.iterrows():
        context = row['combined_text']
        answer = get_answer(question, context)

        if len(answer.strip()) < min_answer_length or answer in ['<s>', '']:
            continue

        sim_score = semantic_similarity(question, answer)
        sim_score = float(sim_score)

        if sim_score > best_score and sim_score >= min_score_threshold:
            best_answer = answer
            best_score = sim_score

    print(best_answer)

    if best_answer:
        return jsonify({'best_answer': best_answer, 'score': best_score})
    else:
        return jsonify({'best_answer': "پاسخ مناسبی یافت نشد."})


In [ ]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://6adb-34-125-245-118.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
[]
     Unnamed: 0                                              title  \
0             0                ‫فلیک به قهرمان بارسلونا پاداش داد‬   
175         175         ‫با دبل ستاره بارسا: اسپانیا نزدیک به طلا‬   
519         519     ‫لوپز با خاکستر کوبایی میرزازاد را برد! (عکس)‬   
517         517  ‫این غول شکست ناپذیر: میرزازاده به امید شانس م...   
110         110             ‫پاداش فوری بارسلونا به ستاره المپیکی‬   

                                               content  \
0    ‫فرمین لوپز با پایان فصل گذشته که از نظر فردی ...   
175  ‫اسپانیا که در دقایق اولیه دیدارش مقابل ‌‏فران...   
519  ‫میخایین لوپز کشتی‌گیر افسانه‌ای کوبا تا دقایق...   
517  ‫در فینال زودرس سنگین وزن کشتی فرنگی، میخایین ...   
110  ‫فرمین لوپز با درخشش در بازی‌های المپیک نشان د...   

                                                  lead  \
0    ‫مربی بارسا به فرمین لوپز یک هفته استراحت داده...   
175

INFO:werkzeug:127.0.0.1 - - [27/Aug/2024 19:27:52] "POST /get_best_answer HTTP/1.1" 200 -


بارسلونا
[]
     Unnamed: 0                                              title  \
0             0                ‫فلیک به قهرمان بارسلونا پاداش داد‬   
175         175         ‫با دبل ستاره بارسا: اسپانیا نزدیک به طلا‬   
110         110             ‫پاداش فوری بارسلونا به ستاره المپیکی‬   
455         455       ‫اصلا فکرش را نمی‌کردم به رکورد نیمار برسم!‬   
499         499  ‫مراکش 1-2 اسپانیا: لاروخا انتقام جام جهانی را...   

                                               content  \
0    ‫فرمین لوپز با پایان فصل گذشته که از نظر فردی ...   
175  ‫اسپانیا که در دقایق اولیه دیدارش مقابل ‌‏فران...   
110  ‫فرمین لوپز با درخشش در بازی‌های المپیک نشان د...   
455  ‫تیم فوتبال المپیک اسپانیا که ابتدا با یک گل ا...   
499  ‫اسپانیا امروز در نیمه‌نهایی فوتبال المپیک ‏دس...   

                                                  lead  \
0    ‫مربی بارسا به فرمین لوپز یک هفته استراحت داده...   
175  ‫فرمین لوپز، ستاره بارسلونا، در فینال المپیک ب...   
110  ‫بارسلونا در روزهای آینده با فرمین وارد

...

اسپانیا
[]
     Unnamed: 0                                              title  \
0             0                ‫فلیک به قهرمان بارسلونا پاداش داد‬   
175         175         ‫با دبل ستاره بارسا: اسپانیا نزدیک به طلا‬   
519         519     ‫لوپز با خاکستر کوبایی میرزازاد را برد! (عکس)‬   
517         517  ‫این غول شکست ناپذیر: میرزازاده به امید شانس م...   
110         110             ‫پاداش فوری بارسلونا به ستاره المپیکی‬   

                                               content  \
0    ‫فرمین لوپز با پایان فصل گذشته که از نظر فردی ...   
175  ‫اسپانیا که در دقایق اولیه دیدارش مقابل ‌‏فران...   
519  ‫میخایین لوپز کشتی‌گیر افسانه‌ای کوبا تا دقایق...   
517  ‫در فینال زودرس سنگین وزن کشتی فرنگی، میخایین ...   
110  ‫فرمین لوپز با درخشش در بازی‌های المپیک نشان د...   

                                                  lead  \
0    ‫مربی بارسا به فرمین لوپز یک هفته استراحت داده...   
175  ‫فرمین لوپز، ستاره بارسلونا، در فینال المپیک ب...   
519  ‫خانواده لوپز، رقیب امین میرزازاد رسومات

INFO:werkzeug:127.0.0.1 - - [27/Aug/2024 19:53:43] "POST /get_best_answer HTTP/1.1" 200 -


اسپانیا
[]
     Unnamed: 0                                              title  \
0             0                ‫فلیک به قهرمان بارسلونا پاداش داد‬   
175         175         ‫با دبل ستاره بارسا: اسپانیا نزدیک به طلا‬   
519         519     ‫لوپز با خاکستر کوبایی میرزازاد را برد! (عکس)‬   
517         517  ‫این غول شکست ناپذیر: میرزازاده به امید شانس م...   
110         110             ‫پاداش فوری بارسلونا به ستاره المپیکی‬   

                                               content  \
0    ‫فرمین لوپز با پایان فصل گذشته که از نظر فردی ...   
175  ‫اسپانیا که در دقایق اولیه دیدارش مقابل ‌‏فران...   
519  ‫میخایین لوپز کشتی‌گیر افسانه‌ای کوبا تا دقایق...   
517  ‫در فینال زودرس سنگین وزن کشتی فرنگی، میخایین ...   
110  ‫فرمین لوپز با درخشش در بازی‌های المپیک نشان د...   

                                                  lead  \
0    ‫مربی بارسا به فرمین لوپز یک هفته استراحت داده...   
175  ‫فرمین لوپز، ستاره بارسلونا، در فینال المپیک ب...   
519  ‫خانواده لوپز، رقیب امین میرزازاد رسومات

INFO:werkzeug:127.0.0.1 - - [27/Aug/2024 19:54:45] "POST /get_best_answer HTTP/1.1" 200 -


بارسلونا


In [ ]:
from pyngrok import ngrok

# توکن احراز هویت Ngrok خود را تنظیم کنید
ngrok.set_auth_token("2kz79ToNRlapXweT40HkWEsKFIr_36TTJ763V4UuxrMyQD8y1")


In [ ]:
public_url = ngrok.connect(5000)
print('Ngrok tunnel "public_url" to port 5000:')
print(public_url)

Ngrok tunnel "public_url" to port 5000:
NgrokTunnel: "https://6f60-35-240-149-177.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
!ngrok config check

Valid configuration file at /root/.config/ngrok/ngrok.yml
